<a href="https://colab.research.google.com/github/AmeerHamza0220/MachineLearningBasic/blob/main/FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle


In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle


In [ ]:
!kaggle competitions download -c fake-news


In [ ]:
!unzip fake-news

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train

In [ ]:
train.dropna(inplace=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
bow_vectorizer = CountVectorizer(max_features=10000)
bow = bow_vectorizer.fit_transform(train['text'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(bow, train['label'], test_size=0.2, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver='lbfgs') 
log_reg.fit(x_train,y_train)
log_reg.score(x_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
ytest_pred=log_reg.predict(x_test)
confusion_matrix(y_test, ytest_pred)


In [ ]:
test.dropna(inplace=True)
test_bow = bow_vectorizer.fit_transform(test['text'])

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
maxlen=1000
vocab_length=5000
def preprocess(data):
    encoded=one_hot(data,vocab_length)
    return encoded
encoded=train['text'].progress_map(preprocess)


In [ ]:
padded=pad_sequences(encoded.to_numpy(),maxlen=maxlen)
X=np.array(padded)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, train['label'], test_size=0.2, random_state=1)

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(vocab_length, 100, input_length=maxlen))
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=64)

In [ ]:
ypred=model.predict(x_test)
ypred